In [37]:
import sched, time
import yfinance as yf
import cryptocompare as cp

In [7]:
import requests as r, json
from getpass import getpass

In [5]:
url = 'https://bigdataforall.com/crypto/?'

In [8]:
print('Enter your bitcoin account key')
key = getpass()

Enter your bitcoin account key
········


In [92]:
profile_url = f"{url}key={key}&request=profile"
reset_url =  f"{url}key={key}&request=reset"
buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"

In [123]:
## reset data
response = r.get(reset_url).json()
## set initial params
qty = 0.1  ## qty planning to buy 
sym = "BTC" 
p_qty = 0 ## purchased qty

In [124]:
response

{'username': 'jodie',
 'cash': 100000,
 'shares value': 0,
 'penalty': 0,
 'total': 100000,
 'shares': []}

In [125]:

while True:
    print ("checking balance and bitcoin price...")
    ## get the current BTC price and current account profile
    BTC = cp.get_price('BTC',currency='USD')['BTC']['USD']
    response = r.get(profile_url).json()
    cash = response['cash']
    total = response['total']
    b_asset = total - cash
    
    print(f"Cash = {cash} , Total = {total}, purchased bitcoin value = {b_asset} /// Bitcion price = {BTC}")
    
    if p_qty == 0: ## first time buy -- buy 55%
        qty = cash*0.55/BTC
        buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
        response = r.get(buy_url).json()
        p_qty = p_qty + qty
        b_cost = cash*0.55
        avg_cost = b_cost / p_qty
        print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    elif avg_cost < BTC: ## if making money: sell 30% owned bitcoin
        if (b_asset/total) > 0.30:
            qty = b_asset*0.30/BTC
            sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"
            response = r.get(sell_url).json()
            p_qty = p_qty - qty
            b_cost = b_cost- (avg_cost*qty)
            print(f"sold {qty} coins, now total own {p_qty} coins")
        else:  ## if owned bitcoin is less than 30% of the total value, then sell 30% but buy 10% back (to increase MA)
            qty = b_asset*0.3/BTC
            sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"
            response = r.get(sell_url).json()
            p_qty = p_qty - qty
            b_cost = b_cost- (avg_cost*qty)
            print(f"sold {qty} coins, now total own {p_qty} coins")
            ## 6% purchase for adjust MA:
            qty = cash*0.1/BTC
            buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
            response = r.get(buy_url).json()
            p_qty = p_qty + qty
            b_cost = b_cost + (cash*0.1)
            avg_cost = b_cost / p_qty
            print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    elif avg_cost > BTC:  ## if not making money: buy 25% cash value bitcoin
        qty = cash*0.25/BTC
        buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
        response = r.get(buy_url).json()
        p_qty = p_qty + qty
        b_cost = b_cost + (cash*0.25)
        avg_cost = b_cost / p_qty
        print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    else:
        print("no action")
    
    
    time.sleep(21)

checking balance and bitcoin price...
Cash = 100000 , Total = 100000, purchased bitcoin value = 0 /// Bitcion price = 54487.53
purchased 1.0094052712611492 coins, now total own 1.0094052712611492 coins
checking balance and bitcoin price...
Cash = 45037.88297983 , Total = 100031.08968235, purchased bitcoin value = 54993.206702519994 /// Bitcion price = 54481.05
purchased 0.2066676531556844 coins, now total own 1.2160729244168336 coins


KeyboardInterrupt: 

In [46]:
# test -- 
BTC = cp.get_price('BTC',currency='USD')['BTC']['USD']

In [47]:
BTC

54691.59